# let's make a datframe split a couple ways by age, then see how harmonization with neurocombat changes it



## import libraries

In [ ]:
import os

import pandas as pd
import numpy as np
from neuroCombat import neuroCombat

## prepare data

In [ ]:
filepath_mri = '../open_work/internal_results/cleaned_pvc2s/' 
filename_mri = os.path.join(filepath_mri,'StrokeMRI_pvc2c.csv') 

In [ ]:
StrokeMRI  = pd.read_csv(filename_mri)
#StrokeMRI = StrokeMRI.drop(StrokeMRI.columns[0],axis=1)
#sex_mapping = {'F':0,'M':1}
#StrokeMRI = StrokeMRI.assign(sex = StrokeMRI.sex.map(sex_mapping))
StrokeMRI.head(3)

In [ ]:
StrokeMRI.age.describe()

Note: StrokeMRI.age.plot() or more importantly  the white matter related values can not be plotted due to environment..

but we must look maybe because 
# ComBat assumes vectors of regression coefficients follow independent normal distributions and some other assumptions

Note: Combat-GAM GMM or Open Nestedmay leapfrog this and give better results, but let's not get ahead

In [ ]:
StrokeMRI.wmh_vol.describe()

In [ ]:
StrokeMRI.wmh_count.describe()

In [ ]:
def log_out_columns(dataframe, column_list): 
    for column in column_list:
        dataframe[column]= np.log(dataframe[column])
    return dataframe

In [ ]:
natural_loged_strokeMRI_unharmonized = log_out_columns(StrokeMRI, ['wmh_vol', 'wmh_count'])

In [ ]:
def split_frame_half_balanced_by_column(frame, column):
    """This is useful in cases where you want to split on a columns
    with continous values e.g. age."""
    df = frame.sort_values(column).reset_index()
    rng_even = range(0, len(df),2)
    rng_odd =  range(1, len(df),2)
    even_rows = df.iloc[rng_even]
    odd_rows = df.iloc[rng_odd]
    return even_rows, odd_rows

In [ ]:
stroke_even_unharmonized = split_frame_half_balanced_by_column(StrokeMRI, 'age')[0]
stroke_odd_unharmonized = split_frame_half_balanced_by_column(StrokeMRI, 'age')[1]

In [ ]:
stroke_even_unharmonized= stroke_even_unharmonized.drop('index', axis=1)
stroke_odd_unharmonized = stroke_odd_unharmonized.drop('index', axis=1)

In [ ]:
stroke_even_unharmonized.to_csv('stroke_even_unharmonized.csv')
stroke_odd_unharmonized.to_csv('stroke_odd_unharmonized.csv') 

In [ ]:
StrokeMRI.head(2)

In [ ]:
stroke_even_unharmonized.head(2)

In [ ]:
#pd.read_csv('stroke_even_unharmonized.csv')

In [ ]:
def top_and_bottom_by_column(frame, column):
    """This is useful in cases where you want to split on a columns
    with continous values e.g. age.; and upi
    want the highest and lowest values seperated"""
    df = frame.sort_values(column)#.reset_index()
    len_first_half= len(df) // 2
    top = df.iloc[:len_first_half]
    bottom = df.iloc[len_first_half :]
    return top, bottom

In [ ]:
stroke_top_unharmonized = top_and_bottom_by_column(StrokeMRI, 'age')[0]
stroke_bottom_unharmonized = top_and_bottom_by_column(StrokeMRI, 'age')[1]

In [ ]:
#stroke_bottom_unharmonized= stroke_bottom_unharmonized.drop('index', axis=1)
#stroke_top_unharmonized= stroke_top_unharmonized.drop('index', axis=1)

In [ ]:
stroke_top_unharmonized.to_csv('stroke_top_unharmonized.csv')
stroke_bottom_unharmonized.to_csv('stroke_bottom_unharmonized.csv') 

In [ ]:
stroke_top_unharmonized.head(3)

In [ ]:
def prep_for_neurocombat(dataframename1, dataframename2):
    """
    This function takes two dataframes in the cvasl format,
    then turns them into three items needed for the 
    neurocombat algorithm with re-identification.
    
    """
    # our_1_frame = pd.read_csv(dataframename1)
    # our_2_frame = pd.read_csv(dataframename2)
    two_selection = dataframename2.drop(['Unnamed: 0'],axis=1)
    one_selection = dataframename1.drop(['Unnamed: 0'],axis=1)
    # two_selection = our_2_frame.drop(['Unnamed: 0.1'],axis=1)
    # one_selection = our_1_frame.drop(['Unnamed: 0.1'],axis=1)
    one_selection = one_selection.set_index('participant_id')
    two_selection = two_selection.set_index('participant_id')
    one_selection = one_selection.T
    two_selection = two_selection.T
    both_togetherF = pd.concat([one_selection, two_selection], axis=1, join="inner")
    print("Nan count",both_togetherF.isna().sum().sum())
    features_only = both_togetherF[2:]
    dictionary_features_len = len(features_only.T.columns)
    number = 0
    made_keys = [] 
    made_vals = [] 
    for n in features_only.T.columns:

        made_keys.append(number)
        made_vals.append(n)
        number +=1
    feature_dictF = dict(map(lambda i,j : (i,j) , made_keys,made_vals))
    ftF = features_only.reset_index()
    ftF = ftF.rename(columns={"index": "A"})
    ftF = ftF.drop(['A'], axis=1)
    ftF = ftF.dropna()
    btF = both_togetherF.reset_index()
    btF = btF.rename(columns={"index": "A"})
    btF = btF.drop(['A'], axis=1)
    btF = btF.dropna()
    len1 = len(one_selection.columns)
    len2 = len(two_selection.columns)
    return both_togetherF, ftF, btF, feature_dictF, len1, len2

In [ ]:
both_togetherF, ftF, btF, feature_dictF, len1, len2 = prep_for_neurocombat(stroke_even_unharmonized, stroke_odd_unharmonized)

In [ ]:
ftF.head(3)

In [ ]:
ftF.to_csv('ftF1_UH.csv')

In [ ]:
data = np.genfromtxt('ftF1_UH.csv', delimiter=",", skip_header=1)
data = data[:, 1:]
data

In [ ]:
# Specifying the batch (scanner variable) as well as a sex covariate to preserve:
first_columns_as_one = [1] * len1
last_columns_as_two = [2] * len2
covars = {'batch':first_columns_as_one + last_columns_as_two,
          'sex':both_togetherF.loc['sex',:].values.tolist(),
           'age':both_togetherF.loc['age',:].values.tolist(),} 
covars = pd.DataFrame(covars) 

In [ ]:
covars

In [ ]:
data

In [ ]:
# specifify sex as categorical
categorical_cols = ['sex']
# to specify the name of the variable that encodes for the scanner/batch covariate:
batch_col_mine = 'batch'
our_continuous_col=['age']
# harmonization step:
data_combat = neuroCombat(dat=data,
    covars=covars,
    batch_col=batch_col_mine,
    continuous_cols=our_continuous_col,
    categorical_cols=categorical_cols)["data"]

In [ ]:
neurocombat = pd.DataFrame(data_combat)
neurocombat.head(2)

In [ ]:
def make_topper(btF, row0,row1):
    """
    This function makes top rows for something harmonized
    out of the btF part produced with prep_for_neurocombat(dataframename1, dataframename2)
    """
    topperF = btF.head(2)
    topperF = topperF.rename_axis(None, axis="columns")
    topperF = topperF.reset_index(drop=False)
    topperF = topperF.rename(columns={"index": "char"})
    topperF['char'][0] = row0#'age'
    topperF['char'][1] = row1#'sex'
    return topperF

In [ ]:
topperF = make_topper(btF,'age', 'sex')

In [ ]:
bottom = neurocombat.reset_index(drop=False)
bottom = bottom.rename(columns={"index": "char"})
bottom.columns = topperF.columns

In [ ]:
back_together = pd.concat([topperF, bottom])
back_together = back_together.T
#back_together

In [ ]:
new_header = back_together.iloc[0] #grab the first row for the header
back_together.columns = new_header #set the header row as the df header
back_together = back_together[1:]
#back_together

In [ ]:
odds_harmonized = back_together.tail(len2)
evens_harmonized = back_together.head(len1)

In [ ]:
#evens

In [ ]:
evens_harmonized = evens_harmonized.rename(feature_dictF, axis='columns')
odds_harmonized= odds_harmonized.rename(feature_dictF, axis='columns')

Save off to harmonized csv files

In [ ]:
evens_harmonized.head(3)

In [ ]:
# evens_harmonized.to_csv('evens_harmonized.csv')
# odds_harmonized.to_csv('odds_harmonized.csv')

Now we can analize the difference between these two sets:

In [ ]:
evens_harmonized.iloc[:,2:] 

In [ ]:
stroke_even_unharmonized.iloc[:,4:] 

## We do see some differences, especially in the wmh volume, but n othing over 100%  at first glance

In [ ]:
both_togetherF2, ftF2, btF2, feature_dictF2, slen1, slen2 = prep_for_neurocombat(stroke_top_unharmonized, stroke_bottom_unharmonized)

In [ ]:
ftF2.to_csv('ftF2_UH.csv')

In [ ]:
data2 = np.genfromtxt('ftF2_UH.csv', delimiter=",", skip_header=1)
data2 = data2[:, 1:]
data2

In [ ]:
# Specifying the batch (scanner variable) as well as a sex covariate to preserve:
first_columns_as_one = [1] * slen1
last_columns_as_two = [2] * slen2
covars2 = {'batch':first_columns_as_one + last_columns_as_two,
          'sex':both_togetherF2.loc['sex',:].values.tolist(),
           'age':both_togetherF2.loc['age',:].values.tolist(),} 
covars2 = pd.DataFrame(covars2) 

In [ ]:
covars2

In [ ]:
# specifify sex as categorical
categorical_cols = ['sex']
# to specify the name of the variable that encodes for the scanner/batch covariate:
batch_col_mine = 'batch'
our_continuous_col=['age']
# harmonization step:
data_combat2 = neuroCombat(dat=data2,
    covars=covars2,
    batch_col=batch_col_mine,
    continuous_cols=our_continuous_col,
    categorical_cols=categorical_cols)["data"]

In [ ]:
neurocombat2 = pd.DataFrame(data_combat2)
neurocombat2.head(2)

In [ ]:
topperF2 = make_topper(btF2,'age', 'sex')

In [ ]:
bottom2 = neurocombat2.reset_index(drop=False)
bottom2 = bottom2.rename(columns={"index": "char"})
bottom2.columns = topperF2.columns

In [ ]:
back_together2 = pd.concat([topperF2, bottom2])
back_together2 = back_together2.T
#back_together2

In [ ]:
new_header2 = back_together2.iloc[0] #grab the first row for the header
back_together2.columns = new_header2 #set the header row as the df header
back_together2 = back_together2[1:]
#back_together2

In [ ]:
bottom_harmonized = back_together2.tail(slen2)
top_harmonized = back_together2.head(slen1)

In [ ]:
top_harmonized = top_harmonized.rename(feature_dictF2, axis='columns')
bottom_harmonized= bottom_harmonized.rename(feature_dictF2, axis='columns')

Save off to harmonized csv files

In [ ]:
top_harmonized = top_harmonized.reset_index()
top_harmonized= top_harmonized.rename(columns= {'index': 'participant_id'})
top_harmonized

In [ ]:
top_harmonized['age'].isna().sum()

In [ ]:
stroke_top_unharmonized

In [ ]:
stroke_top_unharmonized = stroke_top_unharmonized.drop('Unnamed: 0', axis=1)#.iloc[:,4:] 
#stroke_top_unharmonized = stroke_top_unharmonized.reset_index('participant_id')
stroke_top_unharmonized

## So we see in this case we got a ten fold difference on the wmh column...
# This deserves a bit more investigation, and some graphing IMHO
note this shows that algorithms that work well positively (correcting inherently different sets) do nor always work well on the negative test (not over-correcting well on similar or the same datasets )

In [ ]:
#stroke_top_unharmonized.merge(top_harmonized, on= 'participant_id')

In [ ]:
stroke_top_unharmonized['age'].isna().sum()

In [ ]:
top_harmonized['age'].isna().sum()

In [ ]:
#df3 = stroke_top_unharmonized['sex']
df4= pd.DataFrame(stroke_top_unharmonized['age'])
df4['UH']=stroke_top_unharmonized['age']
df4['H']= top_harmonized['age'] 
df4['age_differnce'] = stroke_top_unharmonized['age']- top_harmonized['age']
df4